# B6 AC bridge
Jeg Vet

huangzw29@mail2.sysu.edu.cn

2021/11/20

In [1]:
import pandas as pd
import numpy as np
import sympy as sp

## 1. Capacity

In [2]:
dfc = pd.read_csv('B6-Data/C.csv')
omg = 10000*2*np.pi
C0 = 0.47e-6
R2 = 510

In [3]:
dfc.describe()
R0, R4 = tuple((dfc.describe().loc['mean'])[:2])
SR0, SR4 = tuple((dfc.describe().loc['std'])[:2])
print(R0, R4)
print(SR0, SR4)

11.249999999999998 132.60999999999999
0.11785113019775827 0.42018514437751625


### 1.1 C, R, Z, D

In [4]:
## exp
C = (C0/R2)*R4
RC = R2*R0/R4
ZC = ( (1/(omg*C)**2 + (RC)**2 )**(1/2) )
DC = omg*C0*R0
print(C, RC, ZC, DC)

## Dig
C_dig = 97.526e-9
RC_dig = 52.50
ZC_dig = 171.43
DC_dig = 0.3221

## Sim
dfc_sim = pd.read_csv('B6-Data/C_simu.csv')
C_sim = (C0/R2)*dfc_sim.at[0,'R4']
RC_sim = R2*dfc_sim.at[0,'R0']/dfc_sim.at[0,'R4']
ZC_sim = ( (1/(omg*C_sim))**2 + (RC_sim**2 ))**(1/2)
DC_sim = omg*C0*dfc_sim.at[0,'R0']

1.222092156862745e-07 43.26596787572581 137.23045535353464 0.33222342311712055


In [5]:
C_par = pd.DataFrame({'C':[C, C_dig, C_sim], 
                    'RC':[RC, RC_dig, RC_sim], 
                    'ZC':[ZC, ZC_dig, ZC_sim],
                    'DC':[DC, DC_dig, DC_sim]},
                    index=['exp','dig','sim'])
C_par.loc['exp_dig',] = (C_par.loc['dig',]-C_par.loc['exp',])/C_par.loc['exp',]  
C_par.loc['exp_sim',] = (C_par.loc['sim',]-C_par.loc['exp',])/C_par.loc['exp',]      
C_par

,C,RC,ZC,DC
exp,1.222092e-07,43.265968,137.230455,0.332223
dig,9.752600e-08,52.500000,171.430000,0.322100
sim,9.999020e-08,51.000000,167.141447,0.320411
exp_dig,-2.019751e-01,0.213425,0.249212,-0.030472
exp_sim,-1.818113e-01,0.178756,0.217962,-0.035556


### 1.2 Uncertainty

In [6]:
R0s, R4s = sp.symbols('R0s R4s') 
subs = {'R0s':R0, 'R4s':R4} 

#### 1.2.1 C

In [7]:
F_C = (C0/R2)*R4s
F_C_R0, F_C_R4 = tuple(sp.diff(F_C, i).evalf(subs = subs) for i in ['R0s', 'R4s'])  
print(F_C_R0, F_C_R4)

# S
Sa = ( (SR0*F_C_R0)**2 + (SR4*F_C_R4)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_C_R0)**2 + (u*F_C_R4)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

0 9.21568627450980e-10
Sa=3.87229446779280E-10
Sb=5.32067895135538E-11
S=3.90867761401892E-10


#### 1.2.2 R

In [8]:
F_RC = R2*R0s/R4s
F_RC_R0, F_RC_R4 = tuple(sp.diff(F_RC, i).evalf(subs = subs) for i in ['R0s', 'R4s'])  
print(F_RC_R0, F_RC_R4)

# S
Sa = ( (SR0*F_RC_R0)**2 + (SR4*F_RC_R4)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_RC_R0)**2 + (u*F_RC_R4)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

3.84586381117563 -0.326264745311257
Sa=0.473518803532807
Sb=0.222838634727339
S=0.523332699557639


#### 1.2.3 Z

In [9]:
F_ZC = ( (1/(omg*(C0/R2)*R4s)**2 + (R2*R0s/R4s)**2 )**(1/2) )
F_ZC_R0, F_ZC_R4 = tuple(sp.diff(F_ZC, i).evalf(subs = subs) for i in ['R0s', 'R4s'])  
print(F_ZC_R0, F_ZC_R4)

# S
Sa = ( (SR0*F_ZC_R0)**2 + (SR4*F_ZC_R4)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_ZC_R0)**2 + (u*F_ZC_R4)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

1.21252253867461 -1.03484243536336
Sa=0.457703769007002
Sb=0.0920345908308880
S=0.466865190470037


#### 1.2.4 D

In [10]:
F_DC = omg*C0*R0s
F_DC_R0, F_DC_R4 = tuple(sp.diff(F_DC, i).evalf(subs = subs) for i in ['R0s', 'R4s'])  
print(F_DC_R0, F_DC_R4)

# S
Sa = ( (SR0*F_DC_R0)**2 + (SR4*F_DC_R4)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_DC_R0)**2 + (u*F_DC_R4)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

0.0295309709437441 0
Sa=0.00348025830155740
Sb=0.00170497140238016
S=0.00387545162897092


## L

In [11]:
dfl = pd.read_csv('B6-Data/L.csv')
omg = 10000*2*np.pi
C0 = 0.47e-6
R3 = 510

In [12]:
dfl.describe()
R0, R2 = tuple((dfl.describe().loc['mean'])[:2])
SR0, SR2 = tuple((dfl.describe().loc['std'])[:2])
print(R0, R2)
print(SR0, SR2)

419.96000000000004 82.42
0.08432740427116397 2.4462215762273063


### 2.1 L, R, Z, Q

In [13]:
## exp
L = (C0*R3)*R2
RL = R3*(R2/R0)
ZL = ( ((omg*L)**2 + (RL)**2 )**(1/2) )
QL = omg*C0*R0
print(L, RL, ZL, QL)

## Dig
L_dig = 19.892e-3
RL_dig = 133.87
ZL_dig = 1257.0
QL_dig = 9.3062

## Sim
dfL_sim = pd.read_csv('B6-Data/L_simu.csv')
L_sim = (C0*R3)*dfL_sim.at[0,'R2']
RL_sim = R3*dfL_sim.at[0,'R2']/dfL_sim.at[0,'R0']
ZL_sim = ( ((omg*L_sim))**2 + (RL_sim**2 ))**(1/2)
QL_sim = omg*C0*dfL_sim.at[0,'R0']

0.019756074000000002 100.09096104390893 1245.3395323569212 12.401826557534754


In [14]:
L_par = pd.DataFrame({'L':[L, L_dig, L_sim], 
                    'RL':[RL, RL_dig, RL_sim], 
                    'ZL':[ZL, ZL_dig, ZL_sim],
                    'QL':[QL, QL_dig, QL_sim]},
                    index=['exp','dig','sim'])
L_par.loc['exp_dig',] = (L_par.loc['dig',]-L_par.loc['exp',])/L_par.loc['exp',]  
L_par.loc['exp_sim',] = (L_par.loc['sim',]-L_par.loc['exp',])/L_par.loc['exp',]
L_par.loc['exp_sim',] = (L_par.loc['sim',]-L_par.loc['exp',])/L_par.loc['exp',]       
L_par

,L,RL,ZL,QL
exp,0.019756,100.090961,1245.339532,12.401827
dig,0.019892,133.870000,1257.000000,9.306200
sim,0.019991,100.009405,1260.045446,12.559522
exp_dig,0.006880,0.337483,0.009363,-0.249611
exp_sim,0.011890,-0.000815,0.011809,0.012715


### 2.2 Uncertainty

In [15]:
R0s, R2s = sp.symbols('R0s R2s') 
subs = {'R0s':R0, 'R2s':R2} 

In [16]:
F_L = (C0*R3)*R2s
F_L_R0, F_L_R2 = tuple(sp.diff(F_L, i).evalf(subs = subs) for i in ['R0s', 'R2s'])  
print(F_L_R0, F_L_R2)

# S
Sa = ( (SR0*F_L_R0)**2 + (SR2*F_L_R2)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_L_R0)**2 + (u*F_L_R2)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

0 0.000239700000000000
Sa=0.000586359311821685
Sb=0.0000138390859524753
S=0.000586522602173182


In [17]:
F_RL = R3*(R2s/R0s)
F_RL_R0, F_RL_R2 = tuple(sp.diff(F_RL, i).evalf(subs = subs) for i in ['R0s', 'R2s'])  
print(F_RL_R0, F_RL_R2)

# S
Sa = ( (SR0*F_RL_R0)**2 + (SR2*F_RL_R2)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_RL_R0)**2 + (u*F_RL_R2)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

-0.238334510534120 1.21440137155920
Sa=2.97076282312535
Sb=0.0714510095603133
S=2.97162194735988


In [18]:
F_ZL = ( ((omg*(C0*R3)*R2s)**2 + (R3*(R2s/R0s))**2 )**(1/2) )
F_ZL_R0, F_ZL_R2 = tuple(sp.diff(F_ZL, i).evalf(subs = subs) for i in ['R0s', 'R2s'])  
print(F_ZL_R0, F_ZL_R2)

# S
Sa = ( (SR0*F_ZL_R0)**2 + (SR2*F_ZL_R2)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_ZL_R0)**2 + (u*F_ZL_R2)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

-0.0191555231239963 15.1096764420883
Sa=36.9616165577475
Sb=0.872358277159611
S=36.9719097089355


In [19]:
F_DL = omg*C0*R0s
F_DL_R0, F_DL_R2 = tuple(sp.diff(F_DL, i).evalf(subs = subs) for i in ['R0s', 'R2s'])  
print(F_DL_R0, F_DL_R2)

# S
Sa = ( (SR0*F_DL_R0)**2 + (SR2*F_DL_R2)**2 )**(1/2)
u = 0.1/(3**(1/2)) 
Sb = ( (u*F_DL_R0)**2 + (u*F_DL_R2)**2 )**(1/2)
S = ( (Sa)**2 + (Sb)**2 )**(1/2)
print('Sa={}\nSb={}\nS={}'.format(Sa, Sb, S))

0.0295309709437441 0
Sa=0.00249027012529310
Sb=0.00170497140238016
S=0.00301800808147717
